# Desarrollo de un análisis detallado de las revisiones de Amazon Instant Video
### Examen Parcial de Lógica de Programación
**Tiempo máximo para la realización del caso:** 240 minutos

## Instrucciones

1. Dar un seguimiento secuencial del siguiente caso presentado, resolviendo con detalle los ejercicios planteados.
2. Cargar en el teams la solución en formato `.html` y el archivo de Jupyter Notebook (.ipynb). Para esto, ir a la pestaña `File > Download as > HTML (.html)`. El archivo debe tener el nombre: `Parcial2_Nombre1Apellido1.html` y asunto: *Parcial2 - Lógica de Programación*.
3. El caso está planteado para ser resuelto en un tiempo de 240 minutos, máximo.

Pueden revisarse los casos desarrollados en clase o cualquier recurso adicional que crea pertinente (servicios web, diapositivas, entre otros) para dar solución a este caso.

## Introducción al Caso de Estudio

**Contexto de Negocio.** Usted es un consultor comercial con nuevos clientes que están interesados en analizar las reseñas de sus productos en Amazon (en lugar de Yelp). Quieren responder preguntas comerciales como: "¿Cuáles son los factores más importantes que generan críticas negativas?", "¿Ha habido cambios importantes en la satisfacción / opiniones de los clientes a lo largo del tiempo?", entre otros.

**Problema de Negocio.** Su tarea principal es *explorar los datos dados y utilizar los resultados de su investigación para extraer componentes de texto relevantes que puedan facilitar el análisis posterior y la construcción de modelos*.

**Contexto Analítico.** El conjunto de datos proporcionado tiene una gran cantidad de reseñas relacionadas con películas que se extrajeron de Amazon entre 1996 y 2014. Al explorar el conjunto de datos, se encontrarán rápidamente situaciones familiares discutidas en casos anteriores.

## Cargando los datos de estudio

El conjunto de datos a utilizar contiene alrededor de 36,795 reseñas de videos de Amazon Instant Video, todas obtenidas del sitio web: http://jmcauley.ucsd.edu/data/amazon/. Ejecute la siguiente instrucción en la consola del sistema operativo, bajo el entorno virtual de *LogPro2020* :

`$ pip install pandas`

En este caso encontrarán algunas celdas de código que contienen los componentes algorítmicos básicos para la lectura de archivos y otras operaciones de ayuda. Estas líneas de lectura permiten obtener las revisiones en una estructura de datos:

In [1]:
import pandas as pd
import numpy as np

AIV = pd.read_csv('Data_Parcial2.csv')
AIV = {group[0]: np.array([np.array(group[1][column]) for column in list(group[1].drop('Year', axis=1).columns)]) for group in AIV.groupby('Year')}
AIV

{2006: array([['A2HVL790PBWYTU', 'A2BFIYZYNK54QX', 'A2I6MHMAZZDCRX',
         'AER15RIMV8E6D', 'AQP1VPK16SVWM', 'AJKWF4W7QD4NS',
         'A5TZXWU8AALIC', 'A16QODENBJVUI1', 'A2HVL790PBWYTU',
         'AHNYRCBNPYS9P', 'A2LNW7GRQ205KS', 'ARHEOXJRZX6NI',
         'A328S9RN3U5M68', 'AJKWF4W7QD4NS', 'A1TPW86OHXTXFC',
         'AQP1VPK16SVWM'],
        ["For sheer intensity and plot topsy-turvies, no other show comes close to 24. You'd think the premise (every hour of the 24 episode-long season is played in real-time; thus 24 encapsulates a full day) would get old by now, but, surprisingly, the producers and writers have managed to come up with another riveting, must-see season. In fact, this might be the best season of them all. The thrills and nail-biting suspense (as well as the SPOILERS alert) begin early in the season five opener, with the shocking slaying of President Palmer and assassination attempts on familiar CTU agents (one of which is tragically successful). Jack Bauer (Kiefer Su

Notamos que `AIV_original` es extremadamente largo, así que estamos comprimiendo la celda de salida para que se muestre una parte con fines de análisis. De aquí en adelante trabajaremos con la variable `AIV` completa para analizar las reseñas. Trabajaremos **únicamente** con las funciones `array()`, `len()`, `append()`, `keys()`, `values()`, `copy()`, el acceso por llaves a diccionarios y el acceso por índices y rangos positivos y negativos a las listas y arreglos de `Numpy`. Es permitido la definición de funciones propias.

La variable `AIV` es un diccionario de datos, cuyas llaves están asociadas con el año en el cual fueron publicados los comentarios. Como valores, se encuentran arreglos bidimensionales de `Numpy` que tienen el siguiente contenido:
1. Un arreglo unidimensional con el **ID de cada usuario** que ha publicado el comentario. El ID corresponde a una cadena aleatoria con una cantidad de caracteres homogénea.
2. Un arreglo unidimensional con el **conjunto de comentarios**. La longitud de cada cada comentario es variable. Cada comentario incluye caracteres alfanuméricos y caracteres especiales.
3. Un arreglo unidimensional con la **puntuación que le dan los usuarios a los productos** de Amazon. La puntuación está en el intervalo [1.0, 5.0].
4. Un arreglo unidimensional con el **día de la semana en el cual se publican los comentarios**. El día de la semana está escrito en texto y corresponde a los siete días de la semana, de lunes a domingo.
5. Un arreglo unidimensional con el **día del mes en el cual se publican los comentarios**. Los días del mes se encuentran escritos como número.
6. Un arreglo unidimensional con el **número del mes en el cual se publican los comentarios**.

Cada uno de los datos almacenados en los arreglos unidimensionales corresponde entre sí. Es decir, el dato en la posición 0 del primer arreglo está asociado con el dato 0 del segundo arreglo, con el dato 0 del tercer arreglo, etc.

A nivel general, la estructura es la siguiente:

```
{Year1:
    array[
        [userID1, userID2, ..., userIDn],
        [review1, review2, ..., reviewn],
        [stars1, stars2, ..., starsn],
        [DayName1, DayName2, ..., DayNamen],
        [DayNumber1, DayNumber2, ..., DayNumbern],
        [MonthNumber1, MonthNumber2, ..., MonthNumbern]
    ],
 Year2: 
    array[
        [userID1, userID2, ..., userIDn],
        [review1, review2, ..., reviewn],
        [stars1, stars2, ..., starsn],
        [DayName1, DayName2, ..., DayNamen],
        [DayNumber1, DayNumber2, ..., DayNumbern],
        [MonthNumber1, MonthNumber2, ..., MonthNumbern]
    ],
  ...
  YearK:
     array[
        [userID1, userID2, ..., userIDn],
        [review1, review2, ..., reviewn],
        [stars1, stars2, ..., starsn],
        [DayName1, DayName2, ..., DayNamen],
        [DayNumber1, DayNumber2, ..., DayNumbern],
        [MonthNumber1, MonthNumber2, ..., MonthNumbern]
    ]
} 
```

### (Valor 0.4) Ejercicio 1:

Vamos a responder algunas preguntas básicas:
1. **(Valor 0.1)** ¿Todos los años almacenados en el diccionario presentan una continuidad y existe algún año(s) en el que no hayan almacenados datos? Para esto puede verificar que todos los años almacenados en el diccionario estén presentes como llaves.
2. **(Valor 0.3)** Verifique que la cantidad total de comentarios almacenados sea de 36795.

In [39]:
for año in AIV:
    if len(AIV[año])==0:
        print("vacio como mi corazon")
    else:
        print("lleno")

lleno
lleno
lleno
lleno
lleno
lleno
lleno
lleno
lleno


In [2]:
cant_coment=0
for coment in AIV:
    cant_coment=cant_coment+len(AIV[coment][1])
print("cantidad de comentarios: ",cant_coment)

cantidad de comentarios:  36795


### (Valor 0.3) Ejercicio 2:

Preste atención al siguiente código ejecutado y responda la pregunta planteada en seguida:

In [3]:
{i:len(AIV[i][0]) for i in AIV}

{2006: 16,
 2007: 118,
 2008: 231,
 2009: 362,
 2010: 376,
 2011: 724,
 2012: 2724,
 2013: 15796,
 2014: 16448}

El resultado presentado correponde a:

**(a)** El promedio de comentarios por año almacenados en un diccionario de salida.

**(b)** La desviación estandar de comentarios por año almacenadas en una lista de salida.

**(c)** La variación de comentarios almacenados por año, ubicadas en una lista de salida.

**(d)** La cantidad de comentarios totales almacenados por año, ubicados en un diccionario de salida

**Respuesta:**
   d, de diego
**Justificación:**
   si vemos en el siguiente codigo (el de abajo) estaremos imprimiendo la cantidad de comentarios hechos por año (el codigo de    abajo fue hecho para imprimir la cantidad de comentarios por año)

In [4]:
for coment in AIV:
    print(len(AIV[coment][1]))

16
118
231
362
376
724
2724
15796
16448


### (Valor 0.5) Ejercicio 3:

Calcule y exprese el incremento en la cantidad de comentarios anuales en porcentaje con dos cifras significativas, de tal forma que se obtenga la salida:

```
2006-2007: 637.50%
2007-2008: 95.76%
...
2013-2014: 4.13%
```

In [5]:
bol=True
for coment in AIV:
    if(bol==False):
        a=(100*len(AIV[coment][1]))/acu-100
        print(round(a,2),"%")
        bol=True
    if(bol==True):
        acu=len(AIV[coment][1])
        bol=False

637.5 %
95.76 %
56.71 %
3.87 %
92.55 %
276.24 %
479.88 %
4.13 %


### (Valor 0.5) Ejercicio 4:

Uno de los programadores experto de la empresa realiza la siguiente instrucción sobre el conjunto de datos entregados:

```salida = {i:sum([len(c) for c in AIV[i][1]]) for i in AIV}```

Sin embargo, luego de finalizar su contrato, el programador abandona su cargo sin documentar adecuadamente el código generado, lo que obliga a que ud desarrolle el algoritmo sin utilizar `Dictionary Comprehension` para que el equipo de desarrollo logre comprenderlo adecuadamente. Explique cuál es el objetivo del algoritmo, analizando el resultado obtenido.

In [13]:
sum=0
salida={}
for i in AIV:
    for c in AIV[i][1]:
        sum=sum+len(c)
        salida.update({i:sum})
print(salida)
print("El objetivo del codigo es calcular la cantidad de palabras de los comentarios de cada año")

{2006: 33146, 2007: 231128, 2008: 594845, 2009: 1272253, 2010: 1821953, 2011: 2843886, 2012: 5261415, 2013: 12346181, 2014: 19022536}
El objetivo del codigo es calcular la cantidad de palabras de los comentarios de cada año


**Objetivo del algoritmo:**

**Análisis:**

### (Valor 0.6) Ejercicio 5:

Un programador de su unidad de trabajo quiere calcular la cantidad de usuarios con IDs diferentes que han comentado sobre AWS. Para eso, desarrolla el siguiente código:

```
users = []
for key in AIV:
    for user in __________:
        if user ___ ___ users:
            users.append(________)
print('Cantidad total de usuarios no repetidos: ' + ____(____(users)))
```

Sin embargo, el código se encuentra incompleto, así que le ha pedido a ud completarlo. Complete el código en el espacio correspondiente y muetre la salida del mismo.

In [11]:
users = []
for key in AIV:
    for user in AIV[key][0]:
        if user not in users:
            users.append(user)
print('Cantidad total de usuarios no repetidos: ' + str(len(users)))

Cantidad total de usuarios no repetidos: 5130


### (Valor 0.6) Ejercicio 6:

Encuentre la cantidad promedio de comentarios por usuario (usuarios distintos) para cada año, redondeando a dos cifras, de tal forma que obtenga una salida como la siguiente:
```
{2006: 1.23,
 2007: 1.79,
 ...
 2014: 3.90}
```

In [24]:
prom={}
for año in AIV:
    use=[]
    for user in AIV[año][0]:
        if user not in use:
            use.append(user)
    prom.update({año:round(len(AIV[año][0])/len(use),2)})
print(prom)

{2006: 1.23, 2007: 1.79, 2008: 2.06, 2009: 1.94, 2010: 1.77, 2011: 2.25, 2012: 2.35, 2013: 3.85, 2014: 3.9}


### (Valor 0.6) Ejercicio 7:

Encuentre la cantidad total de comentarios que son puntuados con cada estrella. Debe obtener una salida con la siguiente estructura:
```
1.0: cantidad comentarios totales
2.0: cantidad comentarios totales
 ...
5.0: cantidad comentarios totales
```

In [ ]:
acu=0
estar=[[1.0,0],[2.0,0],[3.0,0],[4.0,0],[5.0,0]]
for año in AIV:
    for i in range(0,len(AIV[año][2])):
        calid=int(AIV[año][2][i])
        estar[calid-1][1]=estar[calid-1][1]+1
for i in range(0,len(estar)):
    print(estar[i][0],": ",estar[i][1])


### (Valor 0.5) Ejercicio 8:

Ordene de mayor a menor la cantidad de comentarios por día de la semana (de lunes a domingo) y analice los resultados. Debe obtener un resultado con la siguiente estructura:
```
Nombre del día Mayor: cantidad comentarios totales
Nombre del siguiente día: cantidad comentarios totales
 ...
Nombre del día Menor: cantidad comentarios totales
```

In [175]:
dia={"Monday":0,"Tuesday":0,"Wednesday":0,"Thursday":0,"Friday":0,"Saturday":0,"Sunday":0}
for año in AIV:
    for i in range(0,len(AIV[año][3])):
        dia1=AIV[año][3][i]
        dia[dia1]=dia[dia1]+1
dias=[];acu=0
for cant in dia:
    dias.append([cant,dia[cant]])
for i in range(0,len(dias)):
    for a in range(0,len(dias)-1):
        if (dias[a][1]<dias[a+1][1]):
            acu=dias[a]
            dias[a]=dias[a+1]
            dias[a+1]=acu
for i in range(0,len(dias)):
    print(dias[i][0],": ",dias[i][1])
       

Sunday :  5622
Thursday :  5394
Tuesday :  5327
Saturday :  5219
Wednesday :  5150
Friday :  5095
Monday :  4988


**Análisis:**

### (Valor 0.5) Ejercicio 9:

Ordene de mayor a menor la cantidad de comentarios por mes (de enero a diciembre) y analice los resultados. Debe obtener un resultado con la siguiente estructura:
```
Nombre del mes Mayor: cantidad comentarios totales
Nombre del siguiente mes: cantidad comentarios totales
 ...
Nombre del mes Menor: cantidad comentarios totales
```

In [176]:
mes=[["Enero",0],["Febrero",0],["Marzo",0],["Abril",0],["Mayo",0],["Junio",0],["Julio",0],["Agosto",0],["Septiembre",0],["Octubre",0],["Noviembre",0],["Diciembre",0]]
for año in AIV:
    for i in range(0,len(AIV[año][5])):
        mes1=int(AIV[año][5][i])
        mes[mes1-1][1]=mes[mes1-1][1]+1
for i in range(0,len(mes)):
    for a in range(0,len(mes)-1):
        if (mes[a][1]<mes[a+1][1]):
            acu=mes[a]
            mes[a]=mes[a+1]
            mes[a+1]=acu
for i in range(0,len(mes)):
    print(mes[i][0],": ",mes[i][1])  

Febrero :  5106
Abril :  4090
Marzo :  4077
Mayo :  3652
Enero :  3391
Julio :  3130
Junio :  2980
Diciembre :  2330
Octubre :  2082
Noviembre :  2034
Septiembre :  2013
Agosto :  1910


**Análisis:**

## Conclusiones

**(Valor 0.3)** Este es un espacio para que escriba las conclusiones del análisis ejecutado en los ejercicios del Parcial Final.

teniendo en cuenta los resultados de los puntos podemos decir que los comentarios fueron en su mayoria positivos, ya que mas de la mitad de los comentarios tienen como puntuacion 4.0 y 5.0, por otro lado, el dia que mas comentarios tuvo es el domingo, yo pensaria que esto pasa por que las personas tienen mas tiempo de escribir sus reseñas este dia ya que es un dia que no se "trabaja" o estudia, respecto a los meses podriamos decir que la gente compra mas (debido a la cantidad de comentarios) a principio de año que finales del año y por ultimo con respecto al incremento del porcentaje de comentarios podemos decir que siempre hay mas comentarios respecto al año pasado, ya que los porcentajes son positivios, si fueran negativos significa que los comentarios disminuyen.

## Aprendizaje del curso

**(Valor 0.2)** Este es un espacio para que escriba una realimentación sobre la materia de Lógica de Programación, en donde mencione lecciones aprendidas, beneficios para su crecimiento personal y oportunidades de mejora en el contenido de la asignatura y en la metodología de enseñanza. Los comentarios son extremadamente importantes para la mejora del proceso docente y el ejercicio profesional. Su honestidad es fundamental para el crecimiento de la asignatura.

profe esta asignatura fue mi favorita, siempre esperaba cona ansias la clase uwu
aprendi 2 lenguajes de programacion nuevos (aunque en php solo se lo basico), siento que mi logica es mucho mas alta respecto a mi yo del año pasado, soy capaz de resolver problemas de una manera mucho mas rapida.